## INSTALLATION AND SYSTEM

In [2]:
import time
import multiprocessing as mp
import pandas as pd

In [3]:
import generate_tasks as gt
import csv_helper as ch
import schedulability as s

In [4]:

! uname -a
print("Number of CPUs:", mp.cpu_count())


Darwin jiwoos-MacBook-Air.local 22.4.0 Darwin Kernel Version 22.4.0: Mon Mar  6 21:01:02 PST 2023; root:xnu-8796.101.5~3/RELEASE_ARM64_T8112 arm64
Number of CPUs: 8


In [5]:

PROCESS_N = mp.cpu_count()
TOTAL_COUNT = 10000

## HELPER FUNCTION

In [16]:
def run_sched_test1_parallel(dataframe, task_sets, indices, num_procs = 1):
  start_time = time.time()
  with mp.Pool(processes = num_procs) as pool:
    sched_test_results = pool.map(s.schedulability_test_thm1, task_sets)
    for i, result in zip(indices, sched_test_results):
      dataframe.loc[i, "thm1"] = result
  end_time = time.time()
  print("Elapsed time {:.2f} s".format(end_time - start_time))

In [17]:
def run_sched_test1(dataframe, t_max_range = range(10), num_procs = 1):
  task_sets = [gt.Task_Set(ts_dict=row.to_dict()) for _, row in dataframe.iterrows() if row["t_max"] in t_max_range and row["thm1"] is None]
  indices = [index for index, row in dataframe.iterrows() if row["t_max"] in t_max_range and row["thm1"] is None]
  if len(task_sets) < 1: 
    print("All done for range {}".format(t_max_range))
    return
  run_sched_test1_parallel(dataframe, task_sets, indices, num_procs)

In [18]:
def run_sched_test23(dataframe, t_max_range = range(10)):
  for index, row in dataframe.iterrows():
    if row["t_max"] in t_max_range and row["thm2"] is None:
      task_set = gt.Task_Set(ts_dict=row.to_dict())
      dataframe.loc[index, "thm2"] = s.schedulability_test_thm2(task_set)
      dataframe.loc[index, "thm3"] =  s.schedulability_test_thm3(task_set)

## TARGET UTILIZATION 500

#### DATAFRAME

In [9]:
TARGET_U_500 = 0.5              
FILENAME_500 = 'task_sets_{}_{}'.format(int(TARGET_U_500 * 1000), TOTAL_COUNT)
FILENAME_500 = 'task_sets_500_10000_v1684186325'

In [10]:
df_500 = ch.read_csv_to_dataframe(FILENAME_500).sort_values('t_max', ascending=True)
print(df_500.describe())
print(df_500['thm1'].isin([True]).sum(), df_500['thm1'].isin([False]).sum())
print(df_500['thm2'].isin([True]).sum(), df_500['thm2'].isin([False]).sum())
print(df_500['thm3'].isin([True]).sum(), df_500['thm3'].isin([False]).sum())


          num_tasks         t_max   utilization
count  10000.000000  1.000000e+04  10000.000000
mean       4.514900  1.261432e+03      0.496835
std        1.001538  1.778127e+04      0.001853
min        3.000000  1.000000e+00      0.485008
25%        4.000000  9.400000e+01      0.496006
50%        4.000000  2.300000e+02      0.497255
75%        5.000000  5.050000e+02      0.498150
max       10.000000  1.192788e+06      0.499806
8468 1338
9956 44
9081 919


#### RUN SCHEDULABILITY TESTS

In [8]:
T_MAX_RANGES_500 = [(int(df_500['t_max'].quantile(i/10)), int(df_500['t_max'].quantile((i+1)/10))) for i in range(0, 10)]
print(T_MAX_RANGES_500)

[(1, 37), (37, 75), (75, 116), (116, 166), (166, 230), (230, 312), (312, 422), (422, 608), (608, 1102), (1102, 1192788)]


In [54]:
for i, j in T_MAX_RANGES_500[:-1]:
  run_sched_test1(df_500, t_max_range = range(i, j), num_procs = PROCESS_N)
  version = int(time.time())
  updated_file_500 = FILENAME_500 + '_v{}'.format(version)
  df_500.to_csv(updated_file_500 + '.csv', index = False)

#### RUN HIGH T_MAX TESTS

In [11]:
FILENAME_500 = 'task_sets_{}_{}'.format(int(TARGET_U_500 * 1000), TOTAL_COUNT)

In [24]:

run_sched_test1(df_500, t_max_range = range(11000), num_procs = PROCESS_N)
version = int(time.time())
updated_file_500 = FILENAME_500 + '_v{}'.format(version)
df_500.to_csv(updated_file_500 + '.csv', index = False)

SpawnPoolWorker-33 8026
SpawnPoolWorker-35 8056
SpawnPoolWorker-34 8067
SpawnPoolWorker-37 8237
SpawnPoolWorker-36 8239
SpawnPoolWorker-38 8239
SpawnPoolWorker-39 8272
SpawnPoolWorker-40 8288
SpawnPoolWorker-37 8407
SpawnPoolWorker-37 8495
SpawnPoolWorker-38 8513
SpawnPoolWorker-35 8554
SpawnPoolWorker-36 8607
SpawnPoolWorker-40 8723
SpawnPoolWorker-39 8749
SpawnPoolWorker-40 8835
SpawnPoolWorker-38 8843
SpawnPoolWorker-38 8877
Elapsed time 1082.95 s


## TARGET UTILIZATION 0.6

#### DATAFRAME

In [57]:
TARGET_U_600 = 0.6             # [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.975]
FILENAME_600 = 'task_sets_{}_{}'.format(int(TARGET_U_600 * 1000), TOTAL_COUNT)

In [65]:
df_600 = ch.read_csv_to_dataframe(FILENAME_600).sort_values('t_max', ascending=True)
print(df_600.describe())
print(df_600['thm1'].isin([True]).sum(), df_600['thm1'].isin([False]).sum())
print(df_600['thm2'].isin([True]).sum(), df_600['thm2'].isin([False]).sum())
print(df_600['thm3'].isin([True]).sum(), df_600['thm3'].isin([False]).sum())


          num_tasks         t_max   utilization
count  10000.000000  1.000000e+04  10000.000000
mean       5.239700  3.953805e+03      0.596149
std        1.071055  1.234986e+05      0.002045
min        3.000000  1.000000e+00      0.583648
25%        4.000000  1.470000e+02      0.595172
50%        5.000000  3.610000e+02      0.596608
75%        6.000000  7.540000e+02      0.597595
max       11.000000  8.297913e+06      0.599736
0 0
0 0
0 0


#### RUN SCHEDULABILITY TESTS

In [60]:
T_MAX_RANGES_600 = [(int(df_600['t_max'].quantile(i/10)), int(df_600['t_max'].quantile((i+1)/10))) for i in range(0, 10)]
print(T_MAX_RANGES_600)

[(1, 59), (59, 114), (114, 183), (183, 263), (263, 361), (361, 482), (482, 641), (641, 896), (896, 1683), (1683, 8297913)]


In [67]:
for i, j in T_MAX_RANGES_600[:-1]:
  run_sched_test23(df_600, t_max_range = range(i, j))
  run_sched_test1(df_600, t_max_range = range(i, j), num_procs = PROCESS_N)
  version = int(time.time())
  updated_file_600 = FILENAME_600 + '_v{}'.format(version)
  df_600.to_csv(updated_file_600 + '.csv', index = False)

SpawnPoolWorker-3975 59
SpawnPoolWorker-3974 60
SpawnPoolWorker-3976 62
SpawnPoolWorker-3977 64
SpawnPoolWorker-3975 59
SpawnPoolWorker-3978 66
SpawnPoolWorker-3979 67
SpawnPoolWorker-3974 60
SpawnPoolWorker-3975 59
SpawnPoolWorker-3978 66
SpawnPoolWorker-3976 62
SpawnPoolWorker-3980 68
SpawnPoolWorker-3979 67
SpawnPoolWorker-3974 60
SpawnPoolWorker-3981 70
SpawnPoolWorker-3977 64
SpawnPoolWorker-3976 62
SpawnPoolWorker-3975 59
SpawnPoolWorker-3978 66
SpawnPoolWorker-3980 68
SpawnPoolWorker-3975 59
SpawnPoolWorker-3977 64
SpawnPoolWorker-3974 60
SpawnPoolWorker-3976 62
SpawnPoolWorker-3978 66
SpawnPoolWorker-3979 67
SpawnPoolWorker-3975 59
SpawnPoolWorker-3974 60
SpawnPoolWorker-3981 70
SpawnPoolWorker-3978 66
SpawnPoolWorker-3976 62
SpawnPoolWorker-3979 67
SpawnPoolWorker-3975 59
SpawnPoolWorker-3974 60
SpawnPoolWorker-3980 69
SpawnPoolWorker-3975 59
SpawnPoolWorker-3981 70
SpawnPoolWorker-3977 64
SpawnPoolWorker-3978 66
SpawnPoolWorker-3976 62
SpawnPoolWorker-3980 69
SpawnPoolWorker-

KeyboardInterrupt: 